# **Barrier (or Interior Point) Method for Linear Programming**


Consider the following Linear Programming problem:
\begin{align*}
(P)\quad \text{min}\quad          & \mathbf{c}^T \mathbf{x} \\
   	\text{ s.t.} \quad	& A \mathbf{x} = \mathbf{b}\\
   		              & \mathbf{x} \geq \mathbf{0}.
\end{align*}
The constraints that make the problem $(P)$ ``hard'' are the variable nonnegativity constraints. We consider applying the logarithmic barrier method (or interior point method) to the problem where only the constraints $\mathbf{x} \geq \mathbf{0}$ are relaxed.
Let $(P(\mu))$ be the logarithmic barrier problem obtained for a given value of the barrier parameter $\mu$, that is:
\begin{align*}
(P(\mu))\quad 	 \text{min}\quad          &\mathbf{c}^T \mathbf{x} - \mu\sum_{i=1}^n{\ln{x_i}} \\
   			\text{ s.t.}\quad & A \mathbf{x} = \mathbf{b}.
\end{align*}


**a**) Determine the Karush-Kuhn-Tucker conditions for the original problem $(P)$. Are they necessary and/or sufficient for the optimality in any point of the feasible region?

Consider the KKT conditions of the problem $(P)$. If we perturb each complementarity condition by a same parameter $\mu >0$, we obtain the following perturbed conditions:
\begin{align}
\mathbf{c} - \mathbf{v} + A^T\mathbf{u} &= \mathbf{0} & &(1)\\
A \mathbf{x} &= \vec{b} & &(2)\\
\mathbf{x} &\geq \mathbf{0} & &(3)\\
\mathbf{v} &\geq \mathbf{0} & & (4)\\
x_j v_j &=\mu & \text{ for } j = 1, \dots, n. & (5)
\end{align}
**Definition**: Let $\left ( \underline{x}\left ( \mu\right ),\underline{u}\left ( \mu\right ),\underline{v}\left ( \mu\right )\right )$ be vectors satisfying the conditions (1)-(5).
As $\mu > 0$ changes, the triplet of vectors $\left ( \underline{x}\left ( \mu\right ),\underline{u}\left ( \mu\right ),\underline{v}\left ( \mu\right )\right )$ describe a trajectory called the *central path*.
Observe that, when $\mu$ tends to $0$, the complementary conditions tends to be satisfied.


More precisely, we can prove that, if $\mu \rightarrow 0$, the triplet $\left ( \underline{x}\left ( \mu\right ),\underline{u}\left ( \mu\right ),\underline{v}\left ( \mu\right )\right )$ converges to a triplet $\left ( \underline{x}^*,\underline{u}^*,\underline{v}^* \right )$ satisfying the KKT conditions of problem $(P)$.

It is possible to verify that
\begin{equation*}
L \left ( \underline{x}\left ( \mu\right ),\underline{u}\left ( \mu\right ),\underline{v}\left ( \mu\right )\right ) = \mathbf{c}^T\mathbf{x}\left ( \mu\right ) - n \mu \le \mathbf{c}^T\mathbf{x}^* \le \mathbf{c}^T\mathbf{x}\left ( \mu\right )
\end{equation*}
where $\mathbf{x}^*$ is an optimal solution for $(P)$ (for the proof, see Appendix 1). Thus, along the central path, for any $\mu > 0$ the value of the objective function of $(P)$ $\mathbf{c}^T\mathbf{x}\left ( \mu\right )$ is at most $n\mu$ from the optimal value $\mathbf{c}^T\mathbf{x}^*$.    

**b**) Consider now the problem with logarithmic barrier $(P(\mu))$ for a given $\mu >0$. Determine the Karush-Kuhn-Tucker conditions for such problem. Are they necessary and/or sufficient for optimality in any point of the feasible region?

**c**) Let $\mathbf{x}^*\left ( \mu \right )$ be a solution satisfying the KKT conditions of $(P(\mu))$ for given multipliers $\mathbf{u}^*\left ( \mu \right )$.
   Can the pair $\left ( \mathbf{x}^*\left ( \mu \right ), \mathbf{u}^*\left ( \mu \right )\right )$ be part of a triplet $\left ( \mathbf{x}^*\left (\mu \right ), \mathbf{u}^*\left ( \mu \right ), \mathbf{v}^*\left ( \mu \right )\right )$ belonging to the central path?

**d**) Consider an iterative algorithm where we apply the Newton method for the solution of the barrier problem at each iteration. Show how one can adapt the rule to derive the Netwon step in order to satisfy a set of linear equality constraints.

**e**) Implement the above-method logarithmic barrier method in Matlab and solve the following problem:
\begin{align}
\text{min}\quad          & x_1 - x_2 & (6)\\
\text{ s.t. } & -x_1 + x_2 \leq 1 & (7)\\
              & x_1 + x_2  \leq 3 & (8)\\
              & x_1, x_2  \geq 0. & (9)
\end{align}, starting from the feasible solution $\bar{\vec{x}}=(1,1)$.

**f**) Solve the problem (6)-(9) with the simplex method, using Python and MIP. Verify the solution found with the interior point method is not on a vertex of the feasible polyhedron, as opposed to the one found by MIP. What is the reason?

In [1]:
import numpy as np
from scipy.optimize import minimize_scalar
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from itertools import combinations

In [2]:
def extrpts(A, rel, b):
    m, n = A.shape
    nlv = n

    for i in range(m):
        e_i = np.zeros(m)
        e_i[i] = 1
        if rel[i] == '>':
            A = np.column_stack([A, -e_i])
        else:
            A = np.column_stack([A, e_i])

        if b[i] < 0:
            A[i, :] = -A[i, :]
            b[i] = -b[i]

    np.seterr(divide='ignore', invalid='ignore')  # Ignore division by zero warnings
    vert = []
    m, n = A.shape
    if n >= m:
        t = list(combinations(range(n), m))
        nv = len(t)

        for i in range(nv):
            y = np.zeros(n)
            x = np.linalg.lstsq(A[:, list(t[i])], b, rcond=None)[0]
            if all(x >= 0) and all(x != np.inf) and all(x != -np.inf):
                y[list(t[i])] = x
                vert.append(y)
    else:
        raise ValueError('Number of equations is greater than the number of variables')

    vert = np.array(vert).T
    #vert = delcols(vert)
    vert = np.unique(vert, axis=1)

    vert = vert[:nlv, :]
    return vert

def polyhedron_print(A, b):
    m, n = A.shape
    Ineq = A[:, :n-m]
    rel = '<' * m
    t = extrpts(Ineq, rel, b)
    t = t[:2, :]
    t = np.unique(t, axis=1) #delcols(t)

    hull = ConvexHull(t.T)
    plt.fill(t[0, hull.vertices], t[1, hull.vertices], color=[0.9, 0.9, 0.5])


In [3]:
# interior point method for LPs of the form min c*x:A*x<=b
def ipm(c, A, b, mu, x_init, gamma, epsilon):
    m, n = A.shape

    # Bring to standard form
    s = b - A.dot(x_init)
    x = np.concatenate([x_init, s])
    A = np.concatenate([A, np.eye(m)], axis=1)
    c = np.concatenate([c, np.zeros(m)])

    grad = np.zeros(m + n)
    H = np.zeros((m + n, m + n))
    d = np.zeros(m + n)
    u = np.zeros(m)

    xks = [x_init]

    while n * mu >= epsilon:
        # compute gradient and Hessian

        # compute direction d with adapted Newton update

        res = minimize_scalar(lambda alpha: c.dot(x + alpha * d) - mu * np.sum(np.log(x + alpha * d)), bounds=(0, 1), method='bounded')
        alpha = res.x

        xstar = x + alpha * d
        mu = gamma * mu
        x = xstar
        xks.append(x[:n])


    vstar = mu / xstar

    polyhedron_print(A, b)
    plt.plot(np.array(xks)[:, 0], np.array(xks)[:, 1], 'r.')
    plt.show()

    print("vstar:", vstar)
    print("xstar:", xstar[:n])

    return xstar[:n], vstar

We use parameters values $\mu_0=1$, $\gamma=0.9$, $\epsilon=0.001$. Recall that $\mathbf{c}=(1,-1,0,0)^T$, $\mathbf{b}=(1,3)^T$ and $\vec{x}(\mu_0)=(1,1)^T$.

To execute the code we use the command:

In [ ]:
ipm(np.array([1, -1 ]), np.array([[-1, 1],[1, 1]]), np.array([1, 3]).T, 1, np.array([1, 1]).T, 0.9, 0.00001)

**g**) Solve the problem obtained by replacing the objective function with $\frac{1}{2} x_1 - x_2$. Verify the solutions of the two methods coincide. Why?

In [ ]:
ipm(np.array([1/2, -1 ]), np.array([[-1, 1],[1, 1]]), np.array([1, 3]).T, 1, np.array([1, 1]).T, 0.9, 0.00001)

By solving with MIP, we obtain the solution $\mathbf{x}^s = (1,2,0,0)^T$. The instruction  **x.rc** prints the reduced costs of the variables in the optimal solution found. The presence of null reduced costs indicates that the optimal solution is not unique (it is possible to execute a change of basis, making pivot on a variable with null reduced cost, obtaining a different solution with equal cost). Analytically, this is due to the fact that the gradient of the objective function is orthogonal to the line $-x_1+x_2=1$, corrsponding to the first constraint. Hence, the polyhedron has an optimal facet, the one connecting the points of the segment between $R=(0,1)$ and $S=(1,2)$. All the points in the segment have the same objective function value, but different values of the barrier term. The solution determined by the interior point method, namely *analytical center*, is the solution that, among all those optimal, minimizes the barrier term.

**Appendix 1: value of the Lagrangian of $(P)$ along the central path**

Let $\left ( \tilde{\mathbf{x}}\left ( \mu \right ), \tilde{\mathbf{u}}\left ( \mu \right ), \tilde{\mathbf{v}}\left ( \mu \right )\right )$ be a solution on the central path. The Lagrangian of $(P)$ is:
\begin{align*}
L\left ( \tilde{\mathbf{x}}\left ( \mu \right ), \tilde{\mathbf{u}}\left ( \mu \right ), \tilde{\mathbf{v}}\left ( \mu \right )\right )= \mathbf{c}^T\tilde{\mathbf{x}}  - \sum_{j=1}^n\tilde{v}_j\tilde{x}_j + \tilde{\mathbf{u}}^T(A\tilde{\mathbf{x}} - \mathbf{b}) = \mathbf{c}^T\tilde{\mathbf{x}} - \sum_{j=1}^n\tilde{v}_j\tilde{x}_j = \mathbf{c}^T\tilde{\mathbf{x}} - n \mu,
\end{align*}
since (1)-(5) are satisfied.

We can also write:
\begin{align*}
L\left ( \tilde{\mathbf{x}}\left ( \mu \right ), \tilde{\mathbf{u}}\left ( \mu\right ), \tilde{\mathbf{v}}\left ( \mu \right ) \right )=
c^T\tilde{\mathbf{x}}  - \sum_{j=1}^n{\tilde{v}}_j{\tilde{x}}_j  =
\left (-A^T\tilde{\mathbf{u}}+\tilde{\mathbf{v}}\right )^T\mathbf{x}  - \sum_{j=1}^n{\tilde{v}}_j{\tilde{x}}_j =
\left (-A^T\tilde{\mathbf{u}}\right )^T{\tilde{\mathbf{x}}} = -\tilde{\mathbf{u}}^TA\tilde{\mathbf{x}} = - \tilde{\mathbf{u}}^T\mathbf{b}
\end{align*}
The Lagrangian of $(P)$ evaluated at a solution on the central path coincides with the value of the objective function of the dual problem.


[ The dual of $(P)$ is as follows:
\begin{align*}
 	\quad\text{max }          & \mathbf{b}^T \mathbf{u} & (D)\\
   		\text{ s.t. } & A^T \mathbf{u} \le \mathbf{c}.
\end{align*}
Since the variables $\mathbf{u}$ are unrestricted in sign, the following problem has the same optimal value of $(D)$:
\begin{align*}
		\quad \text{max }          & -\mathbf{b}^T \mathbf{u} \\
   		 \text{ s.t. } & -A^T \mathbf{u} \le \mathbf{c}.
\end{align*}]


Denoting by $\mathbf{x}^*$ an optimal solution for $(P)$, it follows that
\begin{align*}
c^T\tilde{\mathbf{x}} - n \mu &= - \tilde{\mathbf{u}}^T\mathbf{b} \le c^T\mathbf{x}^* &\text{(due to weak duality in linear programming)}\\
c^T\mathbf{x}^* &\le c^T\tilde{\mathbf{x}}  &\text{(since $c^T\mathbf{x}^*$ is the optimal value)},
\end{align*}
implying that
\begin{align*}
c^T\tilde{\mathbf{x}} - n \mu \le c^T\mathbf{x}^* \le c^T\tilde{\mathbf{x}}
\end{align*}
Then, along the central path, when $\mu$ tends to $0$, the Lagrangian of $(P)$ tends indeed to the optimal vlaue of the objective function.